# § Chapter 12 Advanced pandas

## 12.1 Categorical Data
* Background and Motivation
* Categorical Type in pandas
* Computations with Categoricals
* Categorical Methods
    
## 12.2 Advanced GroupBy Use
* Group Transforms and “Unwrapped” GroupBys
* Grouped Time Resampling
    
## 12.3 Techniques for Method Chaining
* The pipe Method
    
## 12.4 Conclusion


In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## § 12.1 Categorical Data

## Categorical Data: Definition + [Examples, Variables & Analysis]
https://www.formpl.us/blog/categorical-data

### § 12.1.1 Background and Motivation

In [2]:
import numpy as np; import pandas as pd
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)  # 回傳不重複的元素

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [6]:
# 另外一種比較簡捷的儲存的方式：編碼
values = pd.Series([0, 1, 0, 0] * 2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
# 編碼的方式
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

In [8]:
# dim 的編碼，用於被編替後的 values
dim.take(values) # 取出指定的 row index 欄位

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [9]:
# list 的資料也行
dim.take([0, 1, 1, 0])

0     apple
1    orange
1    orange
0     apple
dtype: object

### § 12.1.2 Categorical Type in pandas

In [10]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,5,3.858058
1,1,orange,8,2.612708
2,2,apple,4,2.995627
3,3,apple,7,2.614279
4,4,apple,12,2.990859
5,5,orange,8,3.845227
6,6,apple,5,0.033553
7,7,apple,4,0.425778


In [11]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

In [12]:
# astype()
fruit_cat = df['fruit'].astype('category')  # pandas 對象轉換為指定的 dtype
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [13]:
type(fruit_cat)

pandas.core.series.Series

In [14]:
fruit_cat.index

RangeIndex(start=0, stop=8, step=1)

In [15]:
# 被宣告為 categorial data 之後，這 series 的 value 也不太一樣
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [18]:
# type(df['fruit'].values)

In [19]:
c

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
Categories (2, object): ['apple', 'orange']

In [20]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [21]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [22]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [23]:
# 直接指定一個 list of categorical data
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [24]:
my_categories.categories

Index(['bar', 'baz', 'foo'], dtype='object')

In [25]:
my_categories.codes

array([2, 0, 1, 2, 0], dtype=int8)

In [26]:
# 先提供所有可能的分類，假設從 0 開始編碼
categories = ['foo', 'bar', 'baz']
# 這是對應的一串「編碼後」的資料
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [27]:
# ordinal variable: 這種 categorical data 內涵 numerical 的味道：就是可以排序的
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)  # 讓分類的類別變得有排序
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [29]:
# 如果建置時忘了加 ordered=True，也可以事後再調整
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### § 12.1.3 Computations with Categoricals

In [30]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658])

In [31]:
# 按照個數比例分為幾群
# ordinal type
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [32]:
type(bins)

pandas.core.arrays.categorical.Categorical

In [33]:
# 或是加上可識別的標籤
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [35]:
bins.codes[:20]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3, 3, 0, 2, 2, 3, 3, 0, 1, 3, 1],
      dtype=int8)

In [36]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [37]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

#### Better performance with categoricals

In [38]:
N = 10000000 # 10,000,000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4)) # // 只取商，不取小數

In [39]:
categories = labels.astype('category')

In [40]:
labels.memory_usage()

80000128

In [41]:
categories.memory_usage()

10000332

In [42]:
%time _ = labels.astype('category')

Wall time: 419 ms


### § 12.1.4 Categorical Methods

In [43]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)  
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [44]:
type(cat_s)

pandas.core.series.Series

In [45]:
cat_s.cat

In [47]:
# cat_s.cat?

In [48]:
# CategoricalAccessor
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [49]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [50]:
# 重設 Categorical 的可能值
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [51]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [52]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

In [54]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]  # isin() 回傳是否為目標種類的 bool series
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [53]:
cat_s.isin(['a', 'b'])

0     True
1     True
2    False
3    False
4     True
5     True
6    False
7    False
dtype: bool

In [55]:
# 限定在有用到的 Categorical
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

## one hot encoding
#### Creating dummy variables for modeling

In [56]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [57]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [58]:
pd.get_dummies(cat_s) # one hot encoding

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## § 12.2 Advanced *GroupBy* Use

### §  12.2.1 Group Transforms and "Unwrapped" GroupBys

In Chapter 10 we looked at the *apply* method in grouped operations for performing
transformations. There is another built-in method called *transform*, which is similar
to apply but imposes more constraints on the kind of function you can use:
- It can produce a scalar value to be broadcast to the shape of the group
- It can produce an object of the same shape as the input group
- It must not mutate its input

In [59]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [60]:
# g = df.groupby('key').value
# g

In [66]:
g.groups

{'a': [0, 3, 6, 9], 'b': [1, 4, 7, 10], 'c': [2, 5, 8, 11]}

In [62]:
# df.groupby('key').groups

In [63]:
# df.groupby('key').mean()

In [64]:
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [67]:
# DataFrameGroupBy object
g_p = df.groupby('key')
g_p

In [68]:
g_p.groups

{'a': [0, 3, 6, 9], 'b': [1, 4, 7, 10], 'c': [2, 5, 8, 11]}

In [69]:
# SeriesGroupBy object
g_p.value

In [70]:
g_p.value.groups

{'a': [0, 3, 6, 9], 'b': [1, 4, 7, 10], 'c': [2, 5, 8, 11]}

In [71]:
for name, item in g_p:
    print(name)
    print(item)

a
  key  value
0   a    0.0
3   a    3.0
6   a    6.0
9   a    9.0
b
   key  value
1    b    1.0
4    b    4.0
7    b    7.0
10   b   10.0
c
   key  value
2    c    2.0
5    c    5.0
8    c    8.0
11   c   11.0


### apply() implicitly passes all the columns for each group as a DataFrame to the custom function.
### transform() passes each column for each group individually as a Series to the custom function.

## Difference between apply() and transform() in Pandas
https://towardsdatascience.com/difference-between-apply-and-transform-in-pandas-242e5cf32705

特別注意：
1) transform() returns a Series that has the same length as the input

In [72]:
# 以下的 x 為 groupby 之後的某一群
g.transform(lambda x: x.mean()) # 回傳的長寬為原 dataFrame 長寬， x 為某類的 series

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [73]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [74]:
# x 為 groupby 後分群的向量
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [78]:
# default ascending
pd.Series([2, 3, 4, 5]).rank()

0    1.0
1    2.0
2    3.0
3    4.0
dtype: float64

In [79]:
pd.Series([2, 3, 4, 5]).rank(ascending=False)

0    4.0
1    3.0
2    2.0
3    1.0
dtype: float64

In [75]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [80]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [81]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [83]:
# SeriesGroupBy object
# g

In [85]:
# g.groups

In [87]:
# for tag, item in g:
#     print(tag)
#     print(item)

In [88]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [89]:
# 向量運算
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### § 12.2.2 Grouped Time Resampling

In [90]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [91]:
# [close...)open
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [92]:
# DatetimeIndexResampler object
df.set_index('time').resample('5min')

In [93]:
df.set_index('time').resample('5min').sum()

,value
time,
2017-05-20 00:00:00,10
2017-05-20 00:05:00,35
2017-05-20 00:10:00,60


In [94]:
# 如果要用兩層 index，分別來自不同的 key, 個別再對時間分群

df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [96]:
# 課本上給的這指令舊版的，會報錯
# time_key = pd.TimeGrouper('5min')

### Google 總是可以找到答案
https://stackoverflow.com/questions/57619777/timegrouper-part-of-pandas?noredirect=1&lq=1

Yes, it was deprecated and removed a while back. Use pd.Grouper instead, it does the same thing. 

In [97]:
# 就換成這新的指令，有 groupBy 的味道
time_key = pd.Grouper(freq='5min')

In [98]:
time_key

TimeGrouper(freq=<5 * Minutes>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', origin='start_day')

In [99]:
# 針對 df2，先對 key 的不同的 category 收集 time, 
# 但在收集的時候，我又換了 freq 為為 time_key
# multi index: key, time_key

# 要先設定 .set_index('time') 指出原 df 中為時間的欄位 (還沒有 time_key 的)
resampled = (df2.set_index('time')
             .groupby(['key', time_key]) # 這是兩層的 index
             .sum())
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [100]:
# 改用最原始的 index
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


## § 12.3 Techniques for Method Chaining (方法鏈)

When applying a sequence of transformations to a dataset, you may find yourself creating
numerous temporary variables that are never used in your analysis.

## Using Pandas Method Chaining to improve code readability
https://towardsdatascience.com/using-pandas-method-chaining-to-improve-code-readability-d8517c5626ac

In [101]:
# 課本沒給例子，就是因為少了一個可用的 load_data()，那我就定義一個吧
def load_data():
    df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                       'col1': np.random.randn(12),
                       'col2': np.random.randn(12),
                       'value': np.arange(12.)})
    return df
df = load_data()
df

,key,col1,col2,value
0,a,-0.983505,-0.072052,0.0
1,b,0.930944,0.544066,1.0
2,c,-0.811676,0.323886,2.0
3,a,-1.830156,-1.683325,3.0
4,b,-0.138730,0.526860,4.0
5,c,0.334088,1.858791,5.0
6,a,0.488675,-0.548419,6.0
7,b,-0.178098,-0.279397,7.0
8,c,2.122315,-0.021299,8.0
9,a,0.061192,-0.287990,9.0


我們要比較一步一步串著作，以及整個串在一起的作法：

```python

# 想要做的事...：方法一：一步一步串著作

df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

In [102]:
# 方法一：一步一步串著作
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [103]:
df

,key,col1,col2,value
0,a,0.572796,-1.186623,0.0
1,b,-1.760372,0.334270,1.0
2,c,1.128179,-1.028389,2.0
3,a,1.568606,-0.168496,3.0
4,b,-0.342277,-0.998948,4.0
5,c,-0.009813,-1.531151,5.0
6,a,0.053072,-1.079111,6.0
7,b,-0.041943,-1.097732,7.0
8,c,-1.694312,0.405148,8.0
9,a,-0.362381,-0.227228,9.0


In [104]:
# df2 = df[df['col2'] < 0]
df2

,key,col1,col2,value,col1_demeaned
0,a,0.572796,-1.186623,0.0,-0.014956
2,c,1.128179,-1.028389,2.0,0.540427
3,a,1.568606,-0.168496,3.0,0.980854
4,b,-0.342277,-0.998948,4.0,-0.930029
5,c,-0.009813,-1.531151,5.0,-0.597565
6,a,0.053072,-1.079111,6.0,-0.534680
7,b,-0.041943,-1.097732,7.0,-0.629696
9,a,-0.362381,-0.227228,9.0,-0.950133
10,b,1.127453,-1.442792,10.0,0.539700
11,c,2.183830,-1.422575,11.0,1.596078


In [105]:
df2['col1'].mean()

0.5877522150834494

In [106]:
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
df2

,key,col1,col2,value,col1_demeaned
0,a,0.572796,-1.186623,0.0,-0.014956
2,c,1.128179,-1.028389,2.0,0.540427
3,a,1.568606,-0.168496,3.0,0.980854
4,b,-0.342277,-0.998948,4.0,-0.930029
5,c,-0.009813,-1.531151,5.0,-0.597565
6,a,0.053072,-1.079111,6.0,-0.534680
7,b,-0.041943,-1.097732,7.0,-0.629696
9,a,-0.362381,-0.227228,9.0,-0.950133
10,b,1.127453,-1.442792,10.0,0.539700
11,c,2.183830,-1.422575,11.0,1.596078


In [108]:
# df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
df2['col1_demeaned']

0    -0.014956
2     0.540427
3     0.980854
4    -0.930029
5    -0.597565
6    -0.534680
7    -0.629696
9    -0.950133
10    0.539700
11    1.596078
Name: col1_demeaned, dtype: float64

In [109]:
df2.groupby('key').groups

{'a': [0, 3, 6, 9], 'b': [4, 7, 10], 'c': [2, 5, 11]}

In [110]:
for tag, values in df2.groupby('key'):
    print(tag)
    print(values)

a
  key      col1      col2  value  col1_demeaned
0   a  0.572796 -1.186623    0.0      -0.014956
3   a  1.568606 -0.168496    3.0       0.980854
6   a  0.053072 -1.079111    6.0      -0.534680
9   a -0.362381 -0.227228    9.0      -0.950133
b
   key      col1      col2  value  col1_demeaned
4    b -0.342277 -0.998948    4.0      -0.930029
7    b -0.041943 -1.097732    7.0      -0.629696
10   b  1.127453 -1.442792   10.0       0.539700
c
   key      col1      col2  value  col1_demeaned
2    c  1.128179 -1.028389    2.0       0.540427
5    c -0.009813 -1.531151    5.0      -0.597565
11   c  2.183830 -1.422575   11.0       1.596078


In [111]:
# result = df2.groupby('key').col1_demeaned.std()
result

key
a    0.833390
b    0.776509
c    1.097079
Name: col1_demeaned, dtype: float64

## 以下我們試用以 assign 來貫徹串著作的方法

```python
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)
```

```python

# 直接賦值比較快，但 assign 可以執行一連串方法 
# 整個串在一起

result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())
```

In [112]:
# 方法二：一路串到底
result = (df2.assign(col1_demeaned=df2.col1 - df2.col1.mean())
          .groupby('key')
          .col1_demeaned.std())

In [113]:
result

key
a    0.833390
b    0.776509
c    1.097079
Name: col1_demeaned, dtype: float64

### 從 load_data 開始，一連串 filtering 的動作，串在一個指令中

想做的事...
```python
df = load_data()
df2 = df[df['col2'] < 0]
```

```python
# filtering 可以這麼作

df = (load_data()
      [lambda x: x['col2'] < 0])
```

```python
# 這裏的 x 指的是讀進來的整筆 df，在不同階段反應不同的 df, df2, g 等
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```

In [114]:
df = load_data()
df2 = df[df['col2'] < 0]

In [115]:
df2

,key,col1,col2,value
2,c,-0.459849,-1.034423,2.0
4,b,-0.254742,-2.213133,4.0
8,c,-0.621249,-0.688756,8.0
9,a,-0.799318,-0.815526,9.0
10,b,0.777233,-0.332420,10.0


In [116]:
df = (load_data()
      [lambda x: x['col2'] < 0])

In [117]:
# 因為又執行一次 load_data() 所以結果會不一樣
df

,key,col1,col2,value
0,a,-1.361428,-0.315813,0.0
1,b,-0.669619,-1.869341,1.0
2,c,0.781199,-0.428579,2.0
3,a,-0.395813,-0.017905,3.0
4,b,-0.180737,-1.720285,4.0
7,b,0.329313,-0.489180,7.0
11,c,1.927640,-0.194964,11.0


In [118]:
result = (load_data() # df
          [lambda x: x.col2 < 0] # df2
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean()) # df["col1_demeaned"]
          .groupby('key')
          .col1_demeaned.std())

In [119]:
result

key
a   NaN
c   NaN
Name: col1_demeaned, dtype: float64

### § 12.3 .1 The pipe Method

```python
# 想做的事...

a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

```python
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```

```python
# 這個課本寫錯了
g = df.groupby(['key1', 'key2'])
# df['col1'] = df['col1'] - g.transform('mean') ← 這個寫錯了
df['col1'] = df['col1'] - g['col1'].transform('mean')
```

```python
# for general cases to remove a list of cols
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```

```python
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```

In [131]:
# 我們再建另一個 df 來釋例
df = pd.DataFrame({'key1': ['a', 'b', 'c'] * 4,
                   'key2': ['A', 'B'] * 6,
                   'col1': np.random.randn(12),
                   'col2': np.random.randn(12),
                   'value': np.arange(12.)})

In [132]:
df

,key1,key2,col1,col2,value
0,a,A,-0.408675,-0.496376,0.0
1,b,B,-2.431594,-0.076429,1.0
2,c,A,0.533811,1.208679,2.0
3,a,B,-0.506167,1.052861,3.0
4,b,A,-0.467567,-2.077133,4.0
5,c,B,-0.804300,0.177638,5.0
6,a,A,-1.623669,-1.679772,6.0
7,b,B,-0.105664,0.499993,7.0
8,c,A,-0.546195,-1.382891,8.0
9,a,B,2.013067,0.163755,9.0


In [133]:
g = df.groupby(['key1', 'key2'])

In [134]:
g.groups

{('a', 'A'): [0, 6], ('a', 'B'): [3, 9], ('b', 'A'): [4, 10], ('b', 'B'): [1, 7], ('c', 'A'): [2, 8], ('c', 'B'): [5, 11]}

In [135]:
df['col1'] = df['col1'] - g['col1'].transform('mean')

In [126]:
# df['col1'] = df['col1'] - g.transform('mean')

In [129]:
# g.groups

{('a', 'A'): [0, 6], ('a', 'B'): [3, 9], ('b', 'A'): [4, 10], ('b', 'B'): [1, 7], ('c', 'A'): [2, 8], ('c', 'B'): [5, 11]}

In [136]:
df

,key1,key2,col1,col2,value
0,a,A,0.607497,-0.496376,0.0
1,b,B,-1.162965,-0.076429,1.0
2,c,A,0.540003,1.208679,2.0
3,a,B,-1.259617,1.052861,3.0
4,b,A,0.038079,-2.077133,4.0
5,c,B,-0.075077,0.177638,5.0
6,a,A,-0.607497,-1.679772,6.0
7,b,B,1.162965,0.499993,7.0
8,c,A,-0.540003,-1.382891,8.0
9,a,B,1.259617,0.163755,9.0


In [138]:
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result

In [139]:
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))

In [140]:
result

,key1,key2,col1,col2,value
1,b,B,0.0,-0.076429,1.0
3,a,B,0.0,1.052861,3.0
5,c,B,0.0,0.177638,5.0
6,a,A,0.0,-1.679772,6.0
8,c,A,0.0,-1.382891,8.0
10,b,A,0.0,-1.265204,10.0


## Conclusion

### 3 Easy Ways to Handle Categorical Data -Python
https://medium.com/analytics-vidhya/3-easy-ways-to-handle-categorical-data-python-b43fbd81e227